In [50]:
import pandas as pd
import time
import numpy as np
import warnings

In [2]:
feat = pd.read_csv("clean_data\\features.csv")
net = pd.read_csv("clean_data\\net.csv")

In [3]:
feat.head()

,Unnamed: 0,chr_num,left,right,strand,class
0,1,1,7276,9260,-,ORFs
1,2,1,31152,32984,+,ORFs
2,3,1,33360,34896,+,ORFs
3,4,1,35096,36392,+,ORFs
4,5,1,36544,37328,+,ORFs


In [4]:
net.head()

,Unnamed: 0,pos,value,chr_num,reads,strand
0,1,156,0.481988,1,1.0,+
1,2,5793,0.481988,1,1.0,+
2,3,5871,0.481988,1,1.0,+
3,4,5883,0.481988,1,1.0,+
4,5,5889,0.481988,1,1.0,+


In [5]:
cute_df = pd.DataFrame({"A": [1,2,3],
          "B": [4,5,6]})
for d in cute_df.itertuples():
    print(d)

Pandas(Index=0, A=1, B=4)
Pandas(Index=1, A=2, B=5)
Pandas(Index=2, A=3, B=6)


In [6]:
feat["reads"] = 0
feat.sort_values(by = ["strand", "chr_num", "left"]).iloc[11:20]

,Unnamed: 0,chr_num,left,right,strand,class,reads
8945,8946,1,21566,21850,+,ORF,0
8946,8947,1,21566,21850,+,CDS,0
7275,7276,1,24352,24705,+,XUTs,0
7276,7277,1,24814,25521,+,XUTs,0
7277,7278,1,27075,28174,+,XUTs,0
7278,7279,1,30017,30860,+,XUTs,0
5172,5173,1,30072,30904,+,CUTs,0
1,2,1,31152,32984,+,ORFs,0
8951,8952,1,31567,32940,+,ORF,0


In [120]:
# feat_test = feat.sort_values(by = ["strand", "chr_num", "left"]).iloc[15:20]
feat_test = feat.sort_values(by = ["strand", "chr_num", "left"])
feat_test

,Unnamed: 0,chr_num,left,right,strand,class,reads
8930,8931,1,335,649,+,CDS,0
8931,8932,1,538,792,+,CDS,0
8934,8935,1,2480,2707,+,ORF,0
8935,8936,1,2480,2707,+,CDS,0
6097,6098,1,9368,9600,+,SUTs,0
...,...,...,...,...,...,...,...
5169,5170,16,939043,941219,-,ORFs,0
16460,16461,16,939279,939671,-,ORF,0
16461,16462,16,939279,939671,-,CDS,0
8929,8930,16,939842,941207,-,XUTs,0


In [129]:
# d_test = net.iloc[800:805]
d_test = net.iloc[:10000]
d_test

,Unnamed: 0,pos,value,chr_num,reads,strand
0,1,156,0.481988,1,1.0,+
1,2,5793,0.481988,1,1.0,+
2,3,5871,0.481988,1,1.0,+
3,4,5883,0.481988,1,1.0,+
4,5,5889,0.481988,1,1.0,+
...,...,...,...,...,...,...
9995,9996,62337,2.891927,1,6.0,+
9996,9997,62338,1.445963,1,3.0,+
9997,9998,62343,0.481988,1,1.0,+
9998,9999,62344,0.481988,1,1.0,+


In [130]:
len(cute_df)

3

In [131]:
pd.options.mode.chained_assignment = None
t0 = time.time()
i = 0

for d in d_test.itertuples():
    if i%1000 == 0:
        print(time.time() - t0)
        
#     l_tuples = []
#     for t in feat_test.itertuples():
#         if t.strand == d.strand and t.chr_num == d.chr_num:
#             if t.left < d.pos and t.right > d.pos:
#                 l_tuples.append(t)


    df = feat_test[(feat_test["strand"] == d.strand) & \
             (feat_test["chr_num"] == d.chr_num) & \
             (feat_test["left"] < d.pos) & \
             (feat_test["right"] > d.pos)]
    dists = list(1/(d.pos - df["left"]))
    total = sum(dists)
    for j in range(len(df)):
        feat_test.at[(df.iat[j,0]-1), "reads"] += d.reads * dists[j] / total
        
        
#     dists = []
#     for t in l_tuples:
#         dists.append(1/(d.pos - t.left))
#     total = sum(dists)
#     for i, t in enumerate(l_tuples):
#         feat_test.at[t.Index, "reads"] += d.reads * dists[i] / total

    i += 1

0.008025646209716797
5.044561147689819
10.699479341506958
15.923902750015259
21.57009983062744
27.10252070426941
32.25686311721802
38.42650079727173
44.151015520095825
49.537431955337524


In [132]:
feat_test

,Unnamed: 0,chr_num,left,right,strand,class,reads
8930,8931,1,335,649,+,CDS,0.0
8931,8932,1,538,792,+,CDS,0.0
8934,8935,1,2480,2707,+,ORF,0.0
8935,8936,1,2480,2707,+,CDS,0.0
6097,6098,1,9368,9600,+,SUTs,222.0
...,...,...,...,...,...,...,...
5169,5170,16,939043,941219,-,ORFs,0.0
16460,16461,16,939279,939671,-,ORF,0.0
16461,16462,16,939279,939671,-,CDS,0.0
8929,8930,16,939842,941207,-,XUTs,0.0


In [136]:
# time estimate in hours
((net.shape[0]/10000) * 49.5 / 60) / 60

6.609961875

In [11]:
t0 = time.time()
for d in net.itertuples():
    l_tuples = []
    for t in feat.itertuples():
        if t.strand == d.strand and t.chr_num == d.chr_num:
            if t.left < d.pos and t.right > d.pos:
                l_tuples.append(t)
    dists = []
    for t in l_tuples:
        dists.append(1/(d.pos - t.left))
    total = sum(dists)
    for i, t in enumerate(l_tuples):
        feat.at[t.Index, "reads"] += d.reads * dists[i] / total
t1 = time.time()

KeyboardInterrupt: 

In [ ]:
print(t1-t0)